In [1]:
# !pip install pycocotools --user
!nvidia-smi
import os
import argparse
import numpy as np
import torch
import torch.optim as optim
from torchvision import transforms
from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, Normalizer
from torch.utils.data import DataLoader
from retinanet import coco_eval
from retinanet import csv_eval
import collections
assert torch.__version__.split('.')[0] == '1'

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

LEARNING_RATE = 1e-5
EPOCH = 200
BATCH_SIZE = 4
MODEL_SAVE_PATH = "/workspace/flat_feet/pytorch-retinanet/weight/"

if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

PRETRAINED_CHECK = True
USE_GPU = True

Tue Nov  2 13:26:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:02:00.0 Off |                  N/A |
| 44%   52C    P0    67W / 280W |      0MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 44%   

# 데이터셋(csv format) 의 경로를 설정합니다!


총 3가지 데이터셋 파일이 필요합니다!\
1.학습용 데이터셋 csv포맷\
2.검증용 데이터셋 csv포맷\
3.class 정보를 알려주는 데이터셋 csv포맷

In [3]:
root_dir = "/workspace/flat_feet/dataset/data(all)/"

csv_train_path = root_dir + "annotations_200.csv" 
csv_valid_path = root_dir + "val_annotations_200.csv" 
csv_class_path = root_dir + "classes.csv" 

# 로더를 생성해 줍니다!

!!! 별도로 건들일것 없이 그냥 shift + enter 눌러주시면 됩니다!

In [ ]:
#train data loader
dataset_train = CSVDataset(train_file=csv_train_path, class_list=csv_class_path,
                                   transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))

sampler = AspectRatioBasedSampler(dataset_train, batch_size=BATCH_SIZE, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=0, collate_fn=collater, batch_sampler=sampler)


#validation data loader 
dataset_val = CSVDataset(train_file=csv_valid_path, class_list=csv_class_path,
                                     transform=transforms.Compose([Normalizer(), Resizer()]))

sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=0, collate_fn=collater, batch_sampler=sampler_val)

# back bone 모델을 설정해 줍니다!

원하시는것 고르시면 됩니다! 저는 보통 resnet 101을 사용했습니다!

In [ ]:
#retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=PRETRAINED_CHECK)
#retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=PRETRAINED_CHECK)
#retinanet = model.resnet50(num_classes=dataset_train.num_classes(), pretrained=PRETRAINED_CHECK)
retinanet = model.resnet101(num_classes=dataset_train.num_classes(), pretrained=PRETRAINED_CHECK)
#retinanet = model.resnet152(num_classes=dataset_train.num_classes(), pretrained=PRETRAINED_CHECK)

# 학습에 사용되는 파라미터들을 설정 합니다.

아마 디폴트로 설정해놓은 값 그대로 사용하셔도 무방할것 같습니다.

In [ ]:
use_gpu = USE_GPU
retinanet = retinanet.cuda()
retinanet = torch.nn.DataParallel(retinanet).cuda()

retinanet.training = True
optimizer = optim.Adam(retinanet.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
loss_hist = collections.deque(maxlen=500)

retinanet.train()
retinanet.module.freeze_bn()
print('Num training images: {}'.format(len(dataset_train)))

In [ ]:
valid_map = 0

for epoch_num in range(EPOCH): 
    retinanet.train()
    retinanet.module.freeze_bn()
    epoch_loss = []

    for iter_num, data in enumerate(dataloader_train):
        try:
            optimizer.zero_grad()

            if torch.cuda.is_available():
                classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot']])
            else:
                classification_loss, regression_loss = retinanet([data['img'].float(), data['annot']])

            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()

            loss = classification_loss + regression_loss

            if bool(loss == 0):
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)
            optimizer.step()
            loss_hist.append(float(loss))
            epoch_loss.append(float(loss))
    
            """
            print(
                'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                    epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))
            """
    
            del classification_loss
            del regression_loss
            
        except Exception as e:
            print(e)
            continue


    print('Evaluating dataset')

    mean_map = 0
    mAP = csv_eval.evaluate(dataset_val, retinanet)
    
    for i in range(25):
        mean_map += mAP[i][0]
    
    if(mean_map > valid_map):
        torch.save(retinanet.module, MODEL_SAVE_PATH +'retinanet_{}.pt'.format(epoch_num))
        valid_map = mean_map
        print("#"*60)
        print('model save!!!!!!!_{}.pt'.format(valid_map))
        print("#"*60)
    
    scheduler.step(np.mean(epoch_loss))

retinanet.eval()